In [12]:
import json
import math
import psycopg2
import pandas as pd
import geopandas as geopd
from pyproj import Geod
from shapely import geometry

In [13]:
dbuser = "postgres"
dbpassword = "test"
dbhost = "localhost"
dbport = "5432"
dbname = "postgres"

g = Geod(ellps='WGS84')
cluster_size = 10000

In [14]:
conn = psycopg2.connect("dbname=" + dbname + " user=" + dbuser + " host=" + dbhost + " password=" + dbpassword + " port=" + dbport)

In [15]:
query_string = """SELECT cellid, range, lat, lon FROM public.cell_towers"""
df1 = pd.read_sql_query(query_string, conn)
df1

/var/folders/v9/p9qs1b6n1fl_tr583wmdmt700000gn/T/ipykernel_32416/1302608478.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(query_string, conn)


,cellid,range,lat,lon
0,26041,4,61.452533,10.192254
1,1311,0,59.130426,11.487974
2,1312,0,59.130426,11.487974
3,1313,0,59.130426,11.487974
4,1715,0,59.275232,11.133299
...,...,...,...,...
16185,71795221,0,69.235039,18.010604
16186,71797771,13,69.667181,18.941422
16187,71797781,7000,69.456907,18.473100
16188,71797791,7000,69.644347,18.890971


In [16]:
def boundingBox(lat, lon):
    distance = math.sqrt(2*((cluster_size/2) ** 2))
    top_right_corner = g.fwd(lon, lat, 45, distance)
    bottom_right_corner = g.fwd(lon, lat, 135, distance)
    bottom_left_corner = g.fwd(lon, lat, 225, distance)
    top_left_corner = g.fwd(lon, lat, 315, distance)
    
    p1 = geometry.Point(top_right_corner)
    p2 = geometry.Point(top_left_corner)
    p3 = geometry.Point(bottom_left_corner)
    p4 = geometry.Point(bottom_right_corner)
    
    return geometry.Polygon([p1, p2, p3, p4])

In [17]:
lat = 71.11082723524058
lon = -3.687989120944268

lat_values = []
lon_values = []
towers_values = []
bounding_box_values = []

direction = 1

cur = conn.cursor()

for i in range(50):
    for j in range(50):
        lat_values.append(lat)
        lon_values.append(lon)
        cluster = boundingBox(lat, lon)
        bounding_box_values.append(cluster)
        points = list(bounding_box_values[0].exterior.coords)
        p1_lon = points[0][0]
        p1_lat = points[0][1]
        p2_lon = points[1][0]
        p2_lat = points[1][1]
        p3_lon = points[2][0]
        p3_lat = points[2][1]
        p4_lon = points[3][0]
        p4_lat = points[3][1]
        towers = 0
        cur.execute("SELECT COUNT(*) FROM public.cell_towers WHERE ST_DWithin(ST_Polygon('LINESTRING(%s %s, %s %s, %s %s, %s %s, %s %s)'::geometry, 4326), ST_MakePoint(lon, lat)::geography, 1, false);" % (float(p1_lon), float(p1_lat), float(p2_lon), float(p2_lat), float(p3_lon), float(p3_lat), float(p4_lon), float(p4_lat), float(p1_lon), float(p1_lat)))
        results = cur.fetchone()
        towers = results[0]
        for index, tower in df1.iterrows():
            if tower['range'] > 0:
                tower_range = geometry.Point(tower['lon'], tower['lat'], tower['range'])
                if cluster.intersects(tower_range):
                    towers = towers + 1
        
        towers_values.append(towers)
        p_next = g.fwd(lon, lat, (90 * direction), cluster_size)
        lon = p_next[0]
        lat = p_next[1]
    p_next = g.fwd(lon, lat, 180, cluster_size - 2000)
    lon = p_next[0]
    lat = p_next[1]
    direction = direction * -1

# geo_dict = {}
# geo_dict["type"] = "FeatureCollection"
# geo_dict["features"] = [{"type": "Feature", "geometry": a} for a in [geometry.mapping(b) for b in bounding_box_values]]
# geojson = json.dumps(geo_dict, indent=4)
# # print(geojson)

# with open("bboxes.json", "w") as outfile:
#     outfile.write(geojson)
        

In [18]:
df2 = pd.DataFrame({
    'lat': lat_values,
    'lon': lon_values,
    'towers': towers_values,
    'bounding_box': bounding_box_values,
})

df2

,lat,lon,towers,bounding_box
0,71.110827,-3.687989,0,POLYGON Z ((-3.5493481716575777 71.15558843282...
1,71.110623,-3.411342,0,POLYGON Z ((-3.272702589402721 71.155383713275...
2,71.110418,-3.134698,0,POLYGON Z ((-2.996059893865967 71.155178996107...
3,71.110213,-2.858057,0,POLYGON Z ((-2.7194200849498977 71.15497428132...
4,71.110008,-2.581418,0,POLYGON Z ((-2.4427831625571 71.1547695689185 ...
...,...,...,...,...
2495,67.136933,-1.361426,0,POLYGON Z ((-1.2459362533504537 67.18172455889...
2496,67.136767,-1.591975,0,POLYGON Z ((-1.476486668638529 67.181558329926...
2497,67.136600,-1.822523,0,POLYGON Z ((-1.7070354992236887 67.18139210230...
2498,67.136434,-2.053070,0,POLYGON Z ((-1.9375827451424825 67.18122587601...


In [19]:
df2.to_csv('coverage.csv', encoding='utf-8', index=False)

In [20]:
coverage = []

for index, record in df2.iterrows():
#     if record['towers'] > 0:
        coverage.append({
            'lat': record['lat'],
            'lon': record['lon'],
            'towers': record['towers'],
        })

with open("coverage.json", "w") as outfile:
    json.dump(coverage, outfile)

In [21]:
cur = conn.cursor()
# drop table if exists
cur.execute("DROP TABLE IF EXISTS coverage;")
cur.execute("CREATE TABLE IF NOT EXISTS coverage (id SERIAL PRIMARY KEY, lat float, lon float, towers integer);")
cur.close()
conn.commit()

cur = conn.cursor()
for index, record in df2.iterrows():
    cur.execute("INSERT INTO coverage (lat, lon, towers) VALUES (%s, %s, '%s');" % (float(record['lat']), float(record['lon']), int(record['towers']),))

conn.commit()

In [22]:
conn.close()